# Frequency Bands and Spectrum

This notebook will be focused on the advantages of using RFSoC for Radio and Software Defined Radio (SDR), as well as covering relevent frequency bands with respect to the RFSoC and common uses for such bands. Time will also be spent covering Nyquist zones and how these relate to the relavent frequency bands.

## Introduction to the Radio Frequency (RF) band

## RFSoC Advantages for Radio

## RFSoC Key Features for SDR

## Spectrum Sharing

## Nyquist Zones

## The RF Spectrum

University Of Strathclyde &copy;